## Importing libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

## Reading files

In [3]:
x = pd.read_csv("features_pca95.csv",header=None)   
y = pd.read_csv("labels.csv")

In [4]:
x.shape[0]

1179375

## Importing models

In [5]:
#from sklearn.tree import DecisionTreeRegressor
#import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
#from sklearn.linear_model import ElasticNet
#from sklearn.linear_model import Lasso
#from sklearn.ensemble import RandomForestRegressor
#from sklearn.model_selection import GridSearchCV
#from sklearn.svm import SVR

In [6]:
#xg = xgb.XGBRegressor(objective = "reg:linear", verbosity = 0, random_state=42, max_depth = 10, alpha=10 )
#en = ElasticNet(random_state=42)
#ls = Lasso(random_state=42)
rf = RandomForestRegressor(random_state=42)
#svm = SVR()
#dc = DecisionTreeRegressor(random_state = 42,criterion="mse",min_samples_split=21)

In [7]:
y = np.array(y).flatten()
y

array([   0.,    0.,    0., ..., 1960.,  864.,  610.])

In [5]:
params = {'eta': [0.3], 'gamma': [2,3], 'learning_rate': [0.01]}
#params = dict(selection=['cyclic','random'],alpha=[2,5,7],fit_intercept=[True,False],normalize=[True,False],precompute=[True,False],copy_X=[True,False],warm_start=[True,False],positive=[True,False])
#params = dict(alpha=[2,5,7],tol=[0.1])
'''params = {
    'bootstrap': [True],
    'max_depth': [80],
    'max_features': [5],
    'min_samples_leaf': [2],
    'min_samples_split': [2,3],
    'n_estimators': [50]
}'''
#params = dict(kernel=['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'])

"params = {\n    'bootstrap': [True],\n    'max_depth': [80],\n    'max_features': [5],\n    'min_samples_leaf': [2],\n    'min_samples_split': [2,3],\n    'n_estimators': [50]\n}"

## Gridsearch for hyperparameter tuning

In [7]:
grid = GridSearchCV(estimator = xg, param_grid = params, cv = 4, verbose=1)

In [8]:
grid_result = grid.fit(x,y)

Fitting 4 folds for each of 2 candidates, totalling 8 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed: 10.9min finished


In [9]:
grid.cv_results_

{'mean_fit_time': array([82.15202308, 79.65749151]),
 'std_fit_time': array([3.71092522, 1.0257184 ]),
 'mean_score_time': array([0.79310852, 0.77923697]),
 'std_score_time': array([0.14550437, 0.15188483]),
 'param_eta': masked_array(data=[0.3, 0.3],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_gamma': masked_array(data=[2, 3],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_learning_rate': masked_array(data=[0.01, 0.01],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'eta': 0.3, 'gamma': 2, 'learning_rate': 0.01},
  {'eta': 0.3, 'gamma': 3, 'learning_rate': 0.01}],
 'split0_test_score': array([-9.06049228, -9.06049228]),
 'split1_test_score': array([0.38878431, 0.38878431]),
 'split2_test_score': array([0.78842684, 0.78842684]),
 'split3_test_score': array([0.26733932, 0.26733932]),
 'mean_test_score': array([-1.90398545, -1.90398545]),

In [10]:
grid_result.best_score_

-1.9039854543559025

In [9]:
grid_result.best_params_

{'eta': 0.3,
 'gamma': 2,
 'learning_rate': 0.01,
 'max_depth': 6,
 'min_child_weight': 3}

In [10]:
grid_result.best_params_

{'eta': 0.05, 'gamma': 1}

In [9]:
grid_result.best_params_

{'eta': 0.3, 'gamma': 2, 'learning_rate': 0.01}

## Applying different models

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
from sklearn.metrics import mean_absolute_error 
from sklearn.model_selection import train_test_split
def plot_learning_curves(model, X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    train_errors, val_errors = [], []
    for m in range(1, len(X_train),500):
        model.fit(X_train[:m], y_train[:m])
        y_train_predict = model.predict(X_train[:m])
        y_val_predict = model.predict(X_val)
        train_errors.append(mean_squared_error(y_train_predict, y_train[:m]))
        val_errors.append(mean_squared_error(y_val_predict, y_val))
        plt.plot(np.sqrt(train_errors), "r-+", linewidth=2, label="train")
        plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="val")
        if val_errors[-1]<5000:
            break

## Train-Test splitting

In [10]:
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

In [11]:
X_train.shape

(943500, 22)

In [12]:
model = rf.fit(X_train,y_train)

In [13]:
TR_pred = rf.predict(X_train)
VA_pred = rf.predict(X_val)

## Train error

In [14]:
mean_absolute_error(TR_pred,y_train)

79.65975139374672

In [15]:
from sklearn.metrics import mean_squared_error
mean_squared_error(TR_pred,y_train)

112329.43823116398

## Val error

In [16]:
mean_absolute_error(VA_pred,y_val)

210.90114463169053

In [17]:
from sklearn.metrics import mean_squared_error
mean_squared_error(VA_pred,y_val)

820203.7827951292

## Recording results

Tr,
Va    model

277536.19191424147,
515125.7363729666     xg1

3951362.539418916,
5431680.5998838395   xg maxdeth 3

32196.118173660423,
387410.2293610688  xg maxdep 10

8691.889859796256,
760533.2312282014   xg maxdep 12

53973.0307858834,
195365.11523405922   xg maxdep 9

34035.35073792925,
302275.4458618484   xg maxdep 10 gamma 3

28431.874212088584
167137.88672629336   xg maxdep 10 alpha 3

29919.17168441201
341208.60859821714   xg maxdep 10 alpha 10

38289.082013721454
69724.56893451572     rf  13.55436198198199,33.81292909379969     mae train,val 

27756661.348933015
29683030.238646906     rf maxdep 3

66386581.14540638
70196178.1867919       en

31805.331692067397
105295.56743263677     rf est 50

23697.10030591627
182942.4631038304      rf est 25

84528.53786716668
546022.3156299818       rf pca99     71.65482395336517,189.4821932803392

112329.43823116398
820203.7827951292       rf pca95     79.65975139374672,210.90114463169053

## Saving the Model

In [16]:
import joblib
#joblib.dump(grid_result.best_estimator_,"xg71065.pkl")
joblib.dump(model,"dc94_124abs.pkl")

['dc94_124abs.pkl']

In [17]:
#plot_learning_curves(dc,x,y)